In [1]:
import sys
import subprocess
import os
from collections import defaultdict
import math
import re

from numba import jit
from numba.typed import List
from numba.typed import Dict
import numba


import random
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

import simple_3dviz as viz
from simple_3dviz.window import show as viz_show
from simple_3dviz.utils import render as viz_render


from sklearn.cluster import KMeans

In [2]:
def error(msg):
    print("*** ERROR ***\n", msg, "\n\n"); 
    input("Press Enter to continue...")
    sys.exit(0)

# Loading a .obj Model

In [3]:
def getint(str, linenb):
    """Parse an integer from str, abort on error
       - linenb the line number in the file in case of error"""
    try:
        v = int(str)
        return v
    except:
        error(f"cannot convert [{str}] to int at line {linenb + 1}")

def getfloat(str, linenb):
    """Parse a float from str, abort on error
       - linenb the line number in the file in case of error"""
    try:
        v = float(str)
        return v
    except:
        error(f"cannot convert [{str}] to int at line {linenb + 1}")

def parseFaceTag(f, lena, linenb):
    """Parse a 'v/t/n' tag line in the obj file
       - lena = (nb_vertices, nb_textures, nb_normals) the current count of vertices/textures/normals
       - linenb the line number in the file in case of error"""
    x = f.split("/")
    #x = x[:1] # uncomment to remove texture and normals.     
    if len(x) == 0 or len(x) > 3:
        error(f"wrong face index: {f} at line {linenb+1}")
    if len(x) == 1:
        x += ['', '']
    if len(x) == 2:
        x += ['']            
    for i in range(3): 
        if x[i] == '':
            x[i] = -1
        else:
            v = getint(x[i], linenb)        
            if (v == 0):
                error(f"wrong face index: {f} at line {linenb+1} (index 0)")
            x[i] = (v - 1) if v > 0 else lena[i] + v                       
    return tuple(x)

def splitFace(face):
    """Convert a face [f1,f2,f3,..,fn] to a list of triangles [(f1,f2,f3), (f1,f3,f4), (f1,f4,f5),...]"""
    if (len(face) == 3):        
        return [tuple(face)]
    return [tuple(face[:3])] + splitFace([face[0]] + face[2:])

def rotateTriangleToMin(T):
    """
    Rotate a triangle ( (v0,t0,n0), (v1,t1,n1), (v2,t2,n2) ) so that it begins with the
    minimum value of the vi's 
    """
    m = min(T[0][0], T[1][0], T[2][0])
    if (m == T[0][0]):
        return T
    if (m == T[1][0]):
        return (T[1], T[2], T[0])
    if (m == T[2][0]):
        return (T[2], T[0], T[1])

In [4]:
def cleanup_vertices(vertice, list_obj, digit_round = 8):
    """
    Clean up the 'vertice' list by removing duplicate values and update the list of meshes 'list_obj' 
    """
    nbv = 0
    new_vertice = []
    dver = {}
    for obj in list_obj:
        for n in range(len(obj)):
            T = list(obj[n])
            for k in range(3):
                (jv, jt, jn) = T[k]
                V = np.round(vertice[jv], digit_round)
                tV = tuple(V)
                if tV not in dver: # first encounter with this vertex
                    dver[tV] = nbv
                    new_vertice.append(V)
                    nbv += 1
                T[k] = (dver[tV], jt, jn)
            obj[n] = tuple(T)
    vertice.clear()
    vertice += new_vertice

def cleanup_textures(texture, list_obj, digit_round = 8):
    """
    Clean up the 'vertice' list by removing duplicate values and update the list of meshes 'list_obj' 
    """    
    nbt = 0
    new_texture = []
    dtex = {}
    for obj in list_obj:
        for n in range(len(obj)):
            T = list(obj[n])
            for k in range(3):
                (jv, jt, jn) = T[k]
                if jt >= 0:
                    V = np.round(texture[jt], digit_round)
                    tV = tuple(V)
                    if tV not in dtex: # first encounter with this texture
                        dtex[tV] = nbt
                        new_texture.append(V)
                        nbt += 1
                    T[k] = (jv, dtex[tV], jn)
                else:
                    T[k] = (jv, -1, jn)                    
            obj[n] = tuple(T)
    texture.clear()
    texture += new_texture

def cleanup_normals(normal, list_obj, digit_round = 8):
    """
    Clean up the 'vertice' list by removing duplicate values and update the list of meshes 'list_obj' 
    """    
    nbn = 0
    new_normal = []
    dnor = {}
    for obj in list_obj:
        for n in range(len(obj)):
            T = list(obj[n])
            for k in range(3):
                (jv, jt, jn) = T[k]
                if jn >= 0:
                    V = np.round(normal[jn], digit_round)
                    tV = tuple(V)
                    if tV not in dnor: # first encounter with this normal
                        dnor[tV] = nbn
                        new_normal.append(V)
                        nbn += 1
                    T[k] = (jv, jt, dnor[tV])
                else:
                    T[k] = (jv, jt, -1)                    
            obj[n] = tuple(T)
    normal.clear()
    normal += new_normal

In [5]:
def findBoundingBox(vertices):
    """
    Find the minimum bounding box of a list of vertices
    """
    xmin , ymin, zmin  = vertices[0]
    xmax , ymax, zmax  = vertices[0]
    for (x,y,z) in vertices:
        xmin = min(xmin,x)
        xmax = max(xmax,x)
        ymin = min(ymin,y)
        ymax = max(ymax,y)
        zmin = min(zmin,z)
        zmax = max(zmax,z)        
    return np.array([xmin,ymin,zmin],dtype=float) , np.array([xmax,ymax,zmax],dtype=float)


def findMeshletBoundingBox(vertices, obj, indT):
    """
    Compute the bounding box for a sub-mesh of triangles
    """
    xmin , ymin, zmin  = vertices[obj[indT[0]][0][0]]
    xmax , ymax, zmax  = vertices[obj[indT[0]][0][0]]
    for k in indT:
        for i in range(3):
            (x,y,z) = vertices[obj[k][i][0]]
            xmin = min(xmin,x)
            xmax = max(xmax,x)
            ymin = min(ymin,y)
            ymax = max(ymax,y)
            zmin = min(zmin,z)
            zmax = max(zmax,z)        
    return np.array([xmin,ymin,zmin],dtype=float) , np.array([xmax,ymax,zmax],dtype=float)  #

# def boundingBoxCenter(R):
#     return (R[0] + R[1])/2

def recenterAndRescale(vertices, ask=True):
    """
    Recenter and rescale a list of vertices so that the bouding box fits in [-1,1]^3
    """
    pr = 2
    (xmin,ymin,zmin),(xmax,ymax,zmax) = findBoundingBox(vertices)    
    print(f"\nBounding box of the model [{round(xmin,pr)},{round(xmax,pr)}]x[{round(ymin,pr)},{round(ymax,pr)}]x[{round(zmin,pr)},{round(zmax,pr)}]")
    ans = "y" if ask==False else input("Do you want to translate/resize the model to fit in [-1,1]^3 (y/N) ?")        
    if len(ans) > 0 and (ans.lower())[0] == "y":        
        cx = (xmin + xmax)/2
        cy = (ymin + ymax)/2
        cz = (zmin + zmax)/2
        mx = (xmax -xmin)/2
        my = (ymax -ymin)/2
        mz = (zmax -zmin)/2
        s = max(mx,my,mz)
        for i in range(len(vertices)):            
            x,y,z  = vertices[i]
            vertices[i] = np.array([(x - cx)/s , (y-cy)/s, (z-cz)/s], dtype=float)                    
        (xmin,ymin,zmin),(xmax,ymax,zmax) = findBoundingBox(vertices)        
    if len(ans) > 0 and (ans.lower())[0] == "y":        
        print(f"New bounding box [{round(xmin,pr)},{round(xmax,pr)}]x[{round(ymin,pr)},{round(ymax,pr)}]x[{round(zmin,pr)},{round(zmax,pr)}]\n")
        
    return (np.array([xmin,ymin,zmin],dtype=float) , np.array([xmax,ymax,zmax],dtype=float)) 

In [6]:
def dist(A, B):
    """ Compute the euclidian distance between 2 vectors """
    V = A - B
    return np.sqrt(np.sum(V*V))

def norm(V):
    """ Return the norm of a vector """
    return np.sqrt(np.sum(V*V))    
    
def normVec(V):
    """ Return a normalized vector. Does nothing for 0 """
    norm = np.sqrt(np.sum(V*V))
    return V if (norm == 0) else V/norm

def crossProduct(VA,VB):
    """ Compute the cross product between two vectors """
    return np.cross(VA,VB)

def Vec(VA, VB):
    """ Construct a vector from 2 vertices """
    return VB - VA

In [7]:

def createTrianglesNormals(vertices, triangles, ind):
    """Construct an array for the the center of the triangles indexes by ind."""
    Nr = np.zeros((len(ind),3), dtype=float)
    for n in range(len(ind)):
        T = triangles[ind[n]]
        V1 = np.array(vertices[T[0][0]])
        V2 = np.array(vertices[T[1][0]])
        V3 = np.array(vertices[T[2][0]])
        sU = Vec(V1, V2)
        sV = Vec(V1, V3)
        N = crossProduct(sU,sV)
        N = normVec(N)
        Nr[n,:] = N
    return Nr
    

def createTrianglesCenters(vertices, triangles, ind):
    """Construct an array for the the center of the triangles indexes by ind."""
    Cr = np.zeros((len(ind),3), dtype=float)
    for n in range(len(ind)):
        T = triangles[ind[n]]
        V1 = np.array(vertices[T[0][0]])
        V2 = np.array(vertices[T[1][0]])
        V3 = np.array(vertices[T[2][0]])
        Cr[n,:] = (V1 + V2 + V3)/3
    return Cr

def createGouraudNormals(vertices, normal, obj, ask = True, force_regeneration = False):
    """
    Compute the "gouraud" normals if not already present or if force_regeneration = True
    """
    if force_regeneration:        
        normal = []
    if len(normal) == 0:                
        print("No normal found. ", end="")
        ans = "y" if ask==False else input("Do you want to compute smooth normals (y/n) ?")       
        if len(ans) > 0 and ((ans.lower())[0] == "y" or (ans.lower())[0] == "o"):        
            # create the normals
            normal = [np.array([0.0, 0.0, 0.0], dtype=float)] * len(vertices)            
            for o in obj:
                for T0,T1,T2 in o:   
                    i0 = T0[0]
                    i1 = T1[0]
                    i2 = T2[0]
                    V0 = Vec(vertices[i1] , vertices[i0])
                    V1 = Vec(vertices[i2] , vertices[i1])
                    V2 = Vec(vertices[i0] , vertices[i2])
                    N0 = crossProduct(V2, V0)
                    N1 = crossProduct(V0, V1)
                    N2 = crossProduct(V1, V2)
                    N = N0 + N1 + N2                    
                    normal[i0] = normal[i0] + N
                    normal[i1] = normal[i1] + N
                    normal[i2] = normal[i2] + N                    
            # insert normal indexes in object
            for o in obj:
                for i in range(len(o)):                        
                    T1, T2, T3 = o[i]
                    o[i] = ((T1[0], T1[1], T1[0]),(T2[0], T2[1], T2[0]),(T3[0], T3[1], T3[0]))
        print(f"done.")

    # normalize the normals, even if we do not recompute them
    for i in range(len(normal)):
        normal[i] = normVec(normal[i])        
        n = norm(normal[i])
        if (n < 0.999) or (n > 1.001):
            print(f"error normal {normal[i]} (index {i}) with norm ={n}")
    print(f"number of normals: {len(normal)}")
    return normal  

In [8]:
def loadObjFile(filename, ask_rescale = False, ask_compute_gouraud = True, force_gouraud_regeneration = False):    
    """
    Load an .obj file. 
    
    where
    - vertice : list of verticles (as 3D numpy vectors)
    - texture : list of texture coords (as 3D numpy vectors) may be empty
    - normal  : list of normals (as 3D numpy vectors) may be empty    
    - obj: list of object. Each one being a list of triangles indices
    - tag: list of names of each object
    """

    vertice = []
    texture = []
    normal = []
    obj = []
    tag = []
    
    currentobj = []
    currentname = ""
    currenttagline = 0
    currentnb = 0
    
    print(f"Reading file {filename}.\n")
    try:
        with open(filename, "r") as f:
            lines = list(f)
    except:
        error(f"Cannot open file [{filename}]")
        
    for linenb , line in enumerate(lines): 
        l = line.split()
        if len(l) == 0:
            l = [""] # dummy value
        if l[0] == 'v':
            v = tuple(float(v) for v in l[1:])[:3]
            if len(v) != 3:
                error(f"found wrong vertex [v] (not 3 components) at line {linenb+1}")
            vertice.append(np.array(v, dtype=float))
            
        if l[0] == "vt":
            vt = tuple(float(t) for t in l[1:])[:2]
            if len(vt) != 2:
                error(f"found wrong texture coord [vt] (not 2 components) at line {linenb+1}")
            texture.append(np.array(vt, dtype=float))
           
        if l[0] == "vn":
            vn = tuple(float(n) for n in l[1:])[:3]
            if len(vn) != 3:
                error(f"found wrong normal [vn] (not 3 components) at line {linenb+1}")
            normal.append(np.array(vn, dtype=float))
            
        if l[0] == "o" or l[0] == "g" or l[0] == "usemtl":
            if len(currentobj) > 0:                
                print(f"--- found object {len(obj)+1} starting from tag [{currentname}] at line {currenttagline+1} with {len(currentobj)} triangles.")
                obj.append(currentobj)
                tag.append(currentname)
                currentobj = [] 
                currentname = "" if len(l) == 1 else " ".join(l)
                currenttagline = linenb
            else:
                currenttagline = linenb
                if len(l) >1:
                    currentname += " | " + " ".join(l)
                              
        if l[0] == 'f':
            face = [parseFaceTag(x , [len(vertice), len(texture), len(normal)], linenb) for x in l[1:]]
            currentobj += splitFace(face)
            
    if len(currentobj) > 0:                
                print(f"--- found object {len(obj)+1} starting from tag <{currentname}> at line {currenttagline+1} with {len(currentobj)} triangles.")
                obj.append(currentobj)
                tag.append(currentname)              
                
    if (len(obj) == 0):
        error("no faces found in the file !")

    setvi = set()
    setti = set()
    setni = set() 
    for o in obj:
        for T in o:
            for (v,t,n) in T:
                
                if v < 0:
                    error(f"negative vertex index {v} found in face {T}")
                if v >= len(vertice):
                    error(f"vertex index out of bound {v}/{len(vertice)} in face {T}")   
                setvi.add(v)
                
                if (t < -1) or (t >= len(texture)):
                    error(f"texture index out of bound {v}/{len(texture)} in face {T}")  
                setti.add(t)
                
                if (n < -1) or (n >= len(normal)):
                    error(f"normal index out of bound {v}/{len(normal)} in face {T}")  
                setni.add(n)
                
        for i in range(len(o)):
            o[i] = rotateTriangleToMin(o[i])

    if len(setti) > 1 and -1 in setti:
        error(f"Missing texture indexes for some faces (but not all).")  

    if len(setni) > 1 and -1 in setni:
        error(f"Missing normal indexes for some faces (but not all).")  

    cleanup_vertices(vertice, obj)
    cleanup_textures(texture, obj)
    cleanup_normals(normal, obj)

    print(f"\nTOTAL :")
    print(f"- {len(vertice)} vertices [v]")        
    print(f"- {len(texture)} texture coords [vt]")
    print(f"- {len(normal)} normals [vn]\n")

    BB = recenterAndRescale(vertice, ask_rescale)
    normal = createGouraudNormals(vertice, normal, obj, ask_compute_gouraud, force_gouraud_regeneration)
    
    return {"vertices" : vertice, "textures" : texture, "gouraud_normals" : normal, "list_obj" : obj, "tags" : tag, "nb_meshes" : len(obj), "bounding_box" : BB}

# Mesh extraction from model 

In [9]:
def extractMesh(obj3D, index):

    vertices = obj3D["vertices"]
    textures = obj3D["textures"]
    gouraud_normals = obj3D["gouraud_normals"]
    triangles = obj3D["list_obj"][index]
    name = obj3D["tags"][index]
    
    mesh = {"vertices" : vertices, "textures" : textures, "gouraud_normals" : gouraud_normals, "triangles" : triangles, "name" : name}
    mesh["bounding_box"] = findMeshletBoundingBox(vertices, triangles, list(range(len(triangles))))
    mesh["triangle_normals"] = createTrianglesNormals(vertices, triangles, list(range(len(triangles))))
    mesh["triangle_centers"] = createTrianglesCenters(vertices, triangles, list(range(len(triangles))))
    mesh["meshlets"] = [list(range(len(triangles)))] # single meshlet containing all triangles
    mesh["cones"] = [(np.zeros(3,dtype=float) , np.zeros(3,dtype=float), 1.0) ] # degenerate cone: meshlet always visible. 
    return mesh

# Displaying a model / mesh in 3D

In [32]:
def viz_addmesh(obj, vertices, ind_mesh = None, color_triangles = [0,0,1]):    
    """
    Create a plottable mesh with a given color
    """
    if ind_mesh == None: 
        ind_mesh = list(range(len(obj)))

    color = np.array(color)
    colors = np.ones((len(vertices), 3), dtype=float)
    if len(color.shape) == 1:
        colors =  colors * color
    else:
        for i in ind_mesh:
            colors[obj[i][0][0],:] =  color[i]
            colors[obj[i][1][0],:] =  color[i]
            colors[obj[i][2][0],:] =  color[i]
                    
    m = viz.Mesh.from_faces(vertices,[[obj[i][0][0], obj[i][1][0], obj[i][2][0] ] for i in ind_mesh], colors=colors) 
    return m
    
def viz_addlines(P1, P2, color):


    
def vizu(obj, vertices):

    colors = np.ones((len(vertices), 3))*[1.0, 0.0, 0.0]


    m = viz.Mesh.from_faces(vertices, [[T[0][0], T[1][0], T[2][0] ] for T in obj], colors=c1) 
  #  m2 = viz.Mesh.from_faces(vertices, [[T[0][0], T[1][0], T[2][0] ] for T in obj[1000:2000]], colors=c2) 

    
    viz_show([m1], camera_position=(1.5, 1.5, 1.5), camera_target=(0, 0, 0),light=(60, 160, 60))

                            

SyntaxError: expected ':' (1055892913.py, line 21)

In [36]:
ob = []
ob.append(viz_addmesh(mesh["triangles"], mesh["vertices"], color= [1,0,0])


SyntaxError: incomplete input (2310349406.py, line 2)

In [37]:
viz_show(ob, camera_position=(2, 2, 2), camera_target=(0, 0, 0),light=(60, 160, 60))

In [30]:
color = np.array([[1,2],[1,2]])
len(color.shape)

2

In [12]:
def display3D_sub(list_obj, vertices, camera_pos, normals = None, centers = None):    
    """
    display a list of meshes, all sharing the same vertex set 'vertices'.
    Each mesh is displayed with a different color
    """                              
    # x,y,z arrays for the vertices coords.
    x = np.array([v[0] for v in vertices])
    y = np.array([v[1] for v in vertices])
    z = np.array([v[2] for v in vertices])

    i = []
    j = []
    k = []
    colors = []
    palette = px.colors.qualitative.Dark24
    for n, obj in enumerate(list_obj): 
        if len(obj) > 0:
            colors += [palette[n % 24] for _ in obj]
            i += [u[0][0] for u in obj]
            j += [u[1][0] for u in obj]
            k += [u[2][0] for u in obj]

    camera = dict(up=dict(x=0,y=0,z=1), center=dict(x=0,y=0,z=0), eye=dict(x=camera_pos[0], y=camera_pos[1], z=camera_pos[2]))    

    
    data = []
    data += [go.Mesh3d(x=x, y=y, z=z, i=i, j=j, k=k, opacity=1,facecolor=colors)]
    data += [go.Scatter3d( x=np.array([-1.0,-1.0, -1.0, -1.0, +1.0, +1.0, +1.0, +1.0]), y=np.array([-1.0,-1.0, +1.0, +1.0, -1.0, -1.0, +1.0, +1.0]), z=np.array([-1.0, +1.0, -1.0, +1.0, -1.0, +1.0, -1.0, +1.0]), mode='markers')]

    if normals is not None:
        for n in range(len(normals)): 
            P1 = centers[n]
            P2 = centers[n] + normals[n]*0.2
            #data += [go.Scatter3d(x=[P1[0],P2[0]], y=[P1[1],P2[1]], z=[P1[2],P2[2]], mode='lines')]
    
    fig = go.Figure(data=data)
    
    fig.update_layout(scene_camera=camera, scene = dict(xaxis = dict(nticks=10, range=[-1,1],), yaxis = dict(nticks=10, range=[-1,1],), zaxis = dict(nticks=10, range=[-1,1])), autosize=False, width=800, height=800)    
    fig.show()


def display3DModel(model, camera_pos = None):
    if camera_pos == None: 
        camera_pos = np.array([1.5, 1.5, 1.5])
    display3D_sub(model["list_obj"],  model["vertices"], camera_pos)    

def displayMeshlets(mesh, camera_pos = None):    
    """
    Visualize the meshlets
    """
    
    obj = mesh["triangles"]
    vertices = mesh["vertices"]
    ind_m = mesh["meshlets"]
    cones = mesh["cones"]
    
    obj_meshlets = [ [obj[n] for n in  im] for im in ind_m]
    
    if camera_pos == None: 
        camera_pos = np.array([1.5, 1.5, 1.5])
    vis = []
    nbm = 0
    disp = 0
    tot = 0
    for n in range(len(obj_meshlets)):       
        U = cones[n][0] - camera_pos
        if (np.dot(normVec(cones[n][1]), normVec(U)) <= 0):#cones[n][2]):
            vis.append(obj_meshlets[n])
            disp += len(obj_meshlets[n])
            nbm += 1
        else: 
            vis.append([])
        tot += len(obj_meshlets[n])
    print("showing ", nbm , "/", len(obj_meshlets), " meshes for ", disp, "/", tot," = ", 100*(disp/tot))
    display3D_sub(vis, vertices, camera_pos, mesh["triangle_normals"],mesh["triangle_centers"])      

# Vector cones computation

In [13]:
@jit(nopython=True)
def dist(A, B):
    """ Compute the euclidian distance beetween 2 vectors """
    V = A - B
    return np.sqrt(np.sum(V*V))

@jit(nopython=True)
def normVec(V):
    """ Return a normalized vector. Does nothing for 0 """
    norm = np.sqrt(np.sum(V*V))
    return V if (norm == 0) else V/norm

@jit(nopython=True)
def crossProduct(VA,VB):
    """ Compute the cross product between two vectors """
    return np.cross(VA,VB)

@jit(nopython=True)
def Vec(VA, VB):
    """ Construct a vector from 2 vertices """
    return VB - VA

In [14]:
@jit(nopython=True)
def cosToAngle(a):
    """convert from cosine to angle in degrees"""
    return np.arccos(a)*180/np.pi


@jit(nopython=True)
def angleToCos(deg):
    """convert from angle in degrees to cosine"""
    return np.cos(deg*np.pi/180)


# library used to construct the array: 'pip install icosphere'
# from icosphere import icosphere
# A, _ = icosphere(2)
# improveCone_vico = A.T
improveCone_vico = np.array([[ 0.        ,  0.        ,  0.52573111, -0.52573111,  0.85065081, -0.85065081, -0.        , -0.        , -0.52573111,  0.52573111, -0.85065081,  0.85065081,  0.        ,  0.30901699, -0.30901699, 0.5       , -0.5       ,  0.5       , -0.5       , -0.30901699, 0.30901699,  0.        ,  0.80901699,  0.30901699,  0.80901699, -0.80901699, -0.30901699, -0.80901699,  0.80901699,  1.        , -0.80901699, -1.        , -0.        , -0.30901699,  0.30901699, -0.5       ,  0.5       , -0.5       ,  0.5       ,  0.        , -0.80901699,  0.80901699],
       [ 0.52573111, -0.52573111,  0.85065081,  0.85065081,  0.        , 0.        , -0.52573111,  0.52573111, -0.85065081, -0.85065081, -0.        , -0.        ,  0.        ,  0.80901699,  0.80901699, 0.30901699,  0.30901699, -0.30901699, -0.30901699, -0.80901699, -0.80901699,  1.        ,  0.5       ,  0.80901699,  0.5       , 0.5       ,  0.80901699,  0.5       , -0.5       ,  0.        , -0.5       ,  0.        ,  0.        , -0.80901699, -0.80901699, -0.30901699, -0.30901699,  0.30901699,  0.30901699, -1.        , -0.5       , -0.5       ],
       [ 0.85065081,  0.85065081,  0.        ,  0.        ,  0.52573111, 0.52573111, -0.85065081, -0.85065081, -0.        , -0.        , -0.52573111, -0.52573111,  1.        ,  0.5       ,  0.5       , 0.80901699,  0.80901699,  0.80901699,  0.80901699,  0.5       , 0.5       ,  0.        ,  0.30901699, -0.5       , -0.30901699, 0.30901699, -0.5       , -0.30901699,  0.30901699,  0.        , 0.30901699,  0.        , -1.        , -0.5       , -0.5       , -0.80901699, -0.80901699, -0.80901699, -0.80901699, -0.        , -0.30901699, -0.30901699]])


@jit(nopython=True)
def findCone(normals, ind, tentative_center = None, eps_start = 0.1, eps_stop = 0.001):
    """
    Find the best bounding cone for a subset of normal vectors. 
    input:
    - normals : array of normal vectors 
    - ind : list of indices of the set of normal vectors to consider for the bounding cone
    return (center, a) where 
    - center : center vector of the cone
    - a : max cosine value for all the angles between the center vector and the subset of normals 
    """

    # create the matrix of normals
    L = len(ind)    
            
    if L == 1:
        return (normVec(normals[ind[0]]), 1.0)
    
    N = np.zeros((L,3), dtype=float)
    for k in range(L):
        N[k,:] = normVec(normals[ind[k]])

    # initial center
    if (tentative_center is not None):
        center = tentative_center # use provided tentative_center (must be normalized)
    else:
        center = normVec(np.sum(N, axis=0)) # start with the barycenter

    a = np.min(N.dot(center)) # compute the min cosine angle between the center and the normals
    
    eps =eps_start
    D = improveCone_vico.shape[1] # number of directions to test
    scaling = eps * np.arccos(a) # scaling factor
    
    while(1):        
        # compute the D tentative new centers
        C = center.reshape(3,1) + (scaling * improveCone_vico) # vector of position        
        C /= np.sqrt(np.sum(C*C, axis=0)).reshape(1,D) # normalize it
        
        # make the dot product with all the normals and take the min value among all normals for each tentative center
        A = N.dot(C)
        
        # find the index i and cosine angle ma of the best tentative center
        i = 0
        ma = -2
        for j in range(D):
            aa = np.min(A[:,j])
            if (aa > ma):
                ma = aa
                i = j
        
        if (ma <= a):
            # no improvement
            if (eps <= eps_stop):
                #print(f"end angle ={np.arccos(a) * 180 / np.pi} \t center={center}") 
                return (center, a)
            eps /= 2
            scaling = eps * np.arccos(a) # scaling factor
            #print(f"new eps {eps}")
        else:
            # update center and cosine angle        
            center = C[:, i].copy()
            a = ma
            #print(f"  imp: a={a} \t center={center}") 


In [15]:
@jit(nopython=True)
def coneCosineAngle(c, dir, points):
    """
    Return the cosine angle of the minimal cone with apex 'c' and direction 'dir' that contains all the points 'points'    
    dir must be normalized
    """
    P = points - c
    P /= (np.sqrt(np.sum(P*P, axis=1)).reshape(points.shape[0],1)) # normalize it, may create NaNs but its ok
    U = P.dot(dir)
    mv = 1.0
    for i in range(len(U)): 
        if U[i] < mv: mv = U[i] # do not count NaN here
    return mv



@jit(nopython=True)
def findTranslatedCone(dir, cosine_angle, points, eps_start, eps_min):

    if (points.shape[0] == 1): # single point is easy
        return points[0,:], np.sqrt(1.0-cosine_angle*cosine_angle)

    D = np.zeros((4,3)) # 4 directions orthogonal to dir
    if dir[0] == 0:
        D[0,0] = 1
    else:
        D[0,0] = -dir[1]
        D[0,1] = dir[0]    
    D[1,:] = crossProduct(dir, D[0,:])
    D[2,:] = -D[0,:]
    D[3,:] = -D[1,:]

    B = points[np.argmin(points.dot(dir))]
    C = B.copy()
    a = coneCosineAngle(C, dir, points)
    eps = eps_start
    
    while (a < cosine_angle):         
        C -= dir * eps
        a = coneCosineAngle(C, dir, points)

    while True:
        # move around to improve angle
        bi = -1 
        for i in range(D.shape[0]):
            a2 = coneCosineAngle(C + eps*D[i], dir, points)
            if (a2 > a):
                a = a2
                bi = i
        if (bi != -1):
            C = C + eps*D[bi] # improvement found
        else: 
            if (eps >= eps_min):
                eps /= 2 # try with smaller step
            else:
                # done moving around, finally move forward                 
                eps = eps_start
                while eps >= eps_min:
                    while (coneCosineAngle(C + dir*eps, dir, points) > cosine_angle):
                        C += dir*eps
                    eps /= 2
            
                # compute backfacing cone
                a = min(cosine_angle, coneCosineAngle(C, dir, points))                
                
                h = (B - C).dot(dir)     
                x = h*(2.0 - 1.0/(a*a))
                F = C + x*dir                
                return F, np.sqrt(1.0-a*a)


@jit(nopython=True)
def computeBackFacingCone_sub(triangle_centers, triangle_normals, ind_mesh, eps_start = 1.0, eps_min = 0.0001):

    dir, a = findCone(triangle_normals, ind_mesh, None, eps_start, eps_min)

    if (a > 0.9999): a = 0.9999 # even flat patch have a small angle
    if (a <= 0): # no bounding cone
        return (np.zeros(3,dtype=float) , np.zeros(3,dtype=float), 1.0) # degenerate cone

    points = np.zeros((len(ind_mesh),3), dtype=float)
    for i in range(len(ind_mesh)):
        points[i,:] = triangle_centers[ind_mesh[i]]

    F, angle = findTranslatedCone(dir, a, points, eps_start, eps_min)
    
    return (F, dir, angle)


def computeBackFacingCones(mesh, eps_start = 1.0, eps_min = 0.0001):
    cones = []
    triangle_normals = mesh["triangle_normals"]
    triangle_centers = mesh["triangle_centers"]
    for ind in mesh["meshlets"]:
        cones.append(computeBackFacingCone_sub(triangle_centers, triangle_normals, ind, eps_start, eps_min))
    mesh["cones"] = cones
    
def setDegenerateCones(mesh):
    mesh["cones"] = []
    for i in range(len(mesh["meshlets"])):
        mesh["cones"].append((np.zeros(3,dtype=float) , np.zeros(3,dtype=float), 1.0))
        

# Creating meshlets

In [16]:
def createMeshlets(mesh, alpha=1.0, beta=1.0):
    """
    Split a mesh inot meshlets
    Other parameters:
        - alpha : coefficient for the importance of normals vs positions
        - beta : factor in front of sqrt(N) for the number of meshlets

    Return
        - a list of list of indices representing the meshlet and a list of  
        - a list of (C,a) where 'C' is the center vector of the cone and 'a' the cosine of the angle for the
          visibility cone associated with a meshlet
    """

    obj = mesh["triangles"]
    vertices = mesh["vertices"]

    # construct the array of normals
    #Nr = triangleNormalArray(obj, vertices)
    Nr = mesh["triangle_normals"]

    # x,y,z arrays for the vertices coords.
    x = [v[0] for v in vertices]
    y = [v[1] for v in vertices]
    z = [v[2] for v in vertices]

    # i, j, k arrays for the indices of the triangles
    i = [u[0][0] for u in obj]
    j = [u[1][0] for u in obj]
    k = [u[2][0] for u in obj]


    # position in space R^3 x R^3 of the triangles (input for k-means)
    center = np.array([[(x[i[n]] + x[j[n]] + x[k[n]])/3 , (y[i[n]] + y[j[n]] + y[k[n]])/3, (z[i[n]] + z[j[n]] + z[k[n]])/3, alpha*Nr[n][0], alpha*Nr[n][1], alpha*Nr[n][2] ]  for n in range(len(obj))])

    # perform k-means clustering
    nbclusters = len(obj) # int(beta*np.sqrt(len(obj)))
    print("Splitting the mesh object in ", nbclusters, " meshlets... ")
    kmeans = KMeans(n_clusters=nbclusters, n_init=10, random_state=1)
    kmeans.fit(center)

    # collect the meshlet in a list of list of triangle indices.
    ind_meshlets = [ [] for _ in range(nbclusters) ]
    for n in range(len(obj)):   
        ind_meshlets[kmeans.labels_[n]].append(n)

    mesh["meshlets"] = ind_meshlets
    
    setDegenerateCones(mesh)
    
    print("done !");

    # create the associated list of (C,a) where C is the  cone center vector and a the cosine angle
    # for the bounding cone of all normal vectors of the triangles in a meshlet
    #cones = []
    #for  k, M in enumerate(ind_meshlets):
    #    C, ma = findCone(Nr, M)
    #    R = findMeshletBoundingBox(vertices, obj, M)
    #    cones.append((C,ma, R))        
    #    print(f"meshlet [{k}] with {len(M)} triangles and visibility angle {cosToAngle(ma):.2f} deg")
     

In [17]:
# load the 3D model
model3D = loadObjFile("spot.obj", ask_compute_gouraud = False)

Reading file spot.obj.

--- found object 1 starting from tag <> at line 1 with 5856 triangles.

TOTAL :
- 2930 vertices [v]
- 3225 texture coords [vt]
- 0 normals [vn]


Bounding box of the model [-0.47,0.47]x[-0.74,0.95]x[-0.67,1.05]
New bounding box [-0.55,0.55]x[-0.98,0.98]x[-1.0,1.0]

No normal found. done.
number of normals: 2930


In [18]:
mesh = extractMesh(model3D,0)

In [ ]:
createMeshlets(mesh, alpha=0.0, beta=5.0)

In [ ]:
computeBackFacingCones(mesh)

In [ ]:
displayMeshlets(mesh, camera_pos = [-2,0,0])

In [ ]:
setDegenerateCones(mesh)
displayMeshlets(mesh, camera_pos = [-2,0,0])

In [ ]:

mesh.keys()

In [ ]:
mesh["cones"]

In [ ]:
computeBackFacingCones(mesh)


In [ ]:
mesh.keys()

In [ ]:
mesh["meshlets"][0] = mesh["meshlets"][0][:10]

In [ ]:
computeBackFacingCone(mesh["triangle_centers"], mesh["triangle_normals"], mesh["meshlets"][0])

In [ ]:
cosToAngle(0.8010594715557301)

In [ ]:
cosToAngle(0.5985588003223271)

In [ ]:
normVec(np.array([-0.97855473,  0.15425119,  0.13651818]))

In [ ]:
mesh.keys()

In [ ]:
len(mesh["triangles"])

In [ ]:
mesh.keys()

In [ ]:
mesh["triangle_centers"]

In [ ]:
U = D["a"]
U.append(0)

In [ ]:
normVec(np.array([ 0.66601037, -0.20492703,  0.71724131]))

In [ ]:
Obj3D = namedtuple("Obj3D", ["triangles", "vertices", "texture", "gouraud", ])